In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [1]:
import gdown
!gdown --fuzzy "https://drive.google.com/file/d/19q09IFhfkOOBOXvn_dKhWjILJtjCcsjc/view?usp=drive_link"
!gdown --fuzzy "https://drive.google.com/file/d/1DiBxn8TXdbJqoSw58pYUeaqO3oOKhuQO/view?usp=drive_link"
!gdown --fuzzy "https://drive.google.com/file/d/1Q2yG_CIDvfdGP-fKVPSw979EYgQukjz5/view?usp=drive_link"

Downloading...
From (original): https://drive.google.com/uc?id=19q09IFhfkOOBOXvn_dKhWjILJtjCcsjc
From (redirected): https://drive.google.com/uc?id=19q09IFhfkOOBOXvn_dKhWjILJtjCcsjc&confirm=t&uuid=25cb662e-8826-43ae-b8a0-e2e3f228fefb
To: /home/ck_bonbon/fintech/final_project/EllipticPlusPlus/Transactions Dataset/txs_features.csv
100%|████████████████████████████████████████| 695M/695M [00:14<00:00, 47.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DiBxn8TXdbJqoSw58pYUeaqO3oOKhuQO
To: /home/ck_bonbon/fintech/final_project/EllipticPlusPlus/Transactions Dataset/txs_classes.csv
100%|██████████████████████████████████████| 2.36M/2.36M [00:00<00:00, 57.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Q2yG_CIDvfdGP-fKVPSw979EYgQukjz5
To: /home/ck_bonbon/fintech/final_project/EllipticPlusPlus/Transactions Dataset/txs_edgelist.csv
100%|██████████████████████████████████████| 4.47M/4.47M [00:00<00:00, 17.3MB/s]


In [2]:
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
df_txs_features

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
df_txs_classes

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")
df_txs_edgelist


Transaction features: 



NameError: name 'pd' is not defined

In [ ]:
# 1. 把 feature DataFrame 裡含 NaN 的行丟掉
df_txs_features = df_txs_features.dropna()
# 2. 取出還在的 txId（假設 index 就是 txId）
valid_tx = set(df_txs_features.index)

# 3. 篩邊表：只保留 txId1 與 txId2 都在 valid_tx 裡面的邊
df_txs_edgelist = df_txs_edgelist[
    df_txs_edgelist['txId1'].isin(valid_tx) &
    df_txs_edgelist['txId2'].isin(valid_tx)
].reset_index(drop=True)

In [ ]:

subset = df_txs_classes[['txId', 'class']]

# 2. 用 merge 把 class 欄併到 txs_features
df_txs_features = df_txs_features.merge(
    subset,
    on='txId',     # 以 txId 當作 key
    how='left'     # 保留 txs_features 所有列，對不到的 class 欄會是 NaN
)
#df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [ ]:
# For notmalize augmented features
from sklearn.preprocessing import MinMaxScaler

for column in df_txs_features.columns[-18:-1]:
    feature = np.array(df_txs_features[column]).reshape(-1,1)
    scaler = MinMaxScaler()
    scaler.fit(feature)
    feature_scaled = scaler.transform(feature)
    df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

...

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

In [ ]:
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,0.009891,0.009891,0.009891,0.009891,1.505606e-04,0.009935,0.018790,1.879015e-02,0.009891,3


In [ ]:
df_txs_features = df_txs_features.set_index('txId')
df_txs_features.sort_index()
y = df_txs_features['class']
time_steps = df_txs_features['Time step']
train_mask = time_steps <= 34
test_mask = time_steps > 34
df_txs_features = df_txs_features.drop(columns=['class', 'Time step'])

,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,Local_feature_9,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
txId,,,,,,,,,,,,,,,,,,,,,
1076,48,-0.168500,0.270909,-0.091383,-0.046932,-0.043875,-0.029140,-0.061584,-0.163591,-0.164980,...,6.180084e-05,3.127478e-05,3.127597e-05,6.254833e-05,6.676470e-06,6.164291e-05,1.185148e-04,1.185150e-04,6.238815e-05,3
2534,6,-0.170834,-0.131425,1.018602,0.028105,0.055376,0.054722,-0.061584,-0.163572,-0.167757,...,2.350094e-05,9.975293e-06,5.421152e-06,2.992583e-05,9.815134e-08,2.194331e-05,1.622788e-05,8.509850e-06,2.990209e-05,2
3181,34,1.305212,-0.210553,-1.756361,-0.121970,97.300650,-0.113002,-0.061584,1.348765,1.321754,...,2.057192e-02,2.057192e-02,2.057192e-02,2.057192e-02,5.070912e-07,2.430414e-04,1.592931e-05,5.025955e-06,2.057192e-02,2
3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,-0.166062,...,4.688075e-05,4.688076e-05,4.688195e-05,4.687833e-05,8.301504e-05,3.244988e-05,8.904078e-05,8.904096e-05,4.687265e-05,3
3889,48,-0.086232,-0.101835,-0.646376,-0.121970,17.046997,-0.113002,-0.061584,-0.074885,-0.081943,...,1.207336e-03,1.207336e-03,1.207337e-03,1.207333e-03,5.184866e-08,2.625465e-04,5.313067e-06,7.031674e-07,1.207300e-03,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403203785,28,-0.172978,-0.172527,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.163640,-0.169455,...,7.368421e-08,7.369663e-08,7.488595e-08,7.126406e-08,5.282356e-08,5.251141e-08,1.157616e-07,1.159417e-07,6.146496e-08,3
403234712,28,-0.172669,-0.158783,-1.201369,-0.121970,-0.063725,-0.113002,-0.061584,-0.163323,-0.169142,...,4.387898e-06,4.387910e-06,4.389099e-06,4.385477e-06,2.477037e-05,4.401941e-06,1.660683e-05,1.660701e-05,4.371017e-06,3
403234715,28,-0.172669,-0.158783,-1.201369,-0.121970,-0.063725,-0.113002,-0.061584,-0.163323,-0.169142,...,4.387898e-06,4.387910e-06,4.389099e-06,4.385477e-06,2.477037e-05,4.401941e-06,1.660683e-05,1.660701e-05,4.371017e-06,3


In [ ]:
df_txs_features
y[y == 2] = 0
y[y == 3] = 2

,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,Local_feature_9,Local_feature_10,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
txId,,,,,,,,,,,,,,,,,,,,,
3321,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,-0.166062,-0.049707,...,0.000047,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047
11108,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,-0.133751,-0.049707,...,0.000493,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493
51816,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,-0.166555,-0.049707,...,0.000040,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040
68869,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,-0.118555,0.300047,...,0.000027,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817
89273,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,5.252974,-0.049707,...,0.074805,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194747812,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,0.568363,-0.049707,...,0.010179,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179
194747925,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,0.557528,-0.049707,...,0.010029,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029
194748063,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,0.553434,-0.049707,...,0.009973,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973


In [ ]:
y

txId
3321         2
11108        2
51816        2
68869        0
89273        0
            ..
194747812    2
194747925    2
194748063    2
194748070    2
194835939    2
Name: class, Length: 202804, dtype: int64

In [ ]:
from torch_geometric.data import Data
import torch

ids_from_features = df_txs_features.index.to_series()
ids_from_src      = df_txs_edgelist['txId1']
ids_from_dst      = df_txs_edgelist['txId2']

# 203769
all_ids = pd.unique(pd.concat([ids_from_src, ids_from_dst]))
id_map  = {orig_id: idx for idx, orig_id in enumerate(all_ids)}

src = df_txs_edgelist['txId1'].map(id_map).values
dst = df_txs_edgelist['txId2'].map(id_map).values
edge_index = torch.tensor([src, dst], dtype=torch.long)

len(all_ids)
df_txs_features.index = df_txs_features.index.map(id_map)
df_txs_features

23

,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,Local_feature_9,Local_feature_10,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
txId,,,,,,,,,,,,,,,,,,,,,
NaN,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,-0.166062,-0.049707,...,0.000047,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047
NaN,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,-0.133751,-0.049707,...,0.000493,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493
NaN,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,-0.166555,-0.049707,...,0.000040,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040
NaN,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,-0.118555,0.300047,...,0.000027,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817
NaN,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,5.252974,-0.049707,...,0.074805,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,0.568363,-0.049707,...,0.010179,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179
NaN,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,0.557528,-0.049707,...,0.010029,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029
NaN,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,0.553434,-0.049707,...,0.009973,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973


In [ ]:
x = torch.tensor(df_txs_features.values, dtype=torch.float)
data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)
data
data.y = torch.from_numpy(data.y.to_numpy().astype(int)).long()

Data(x=[202804, 182], edge_index=[2, 22], y=txId
3321         2
11108        2
51816        2
68869        0
89273        0
            ..
194747812    2
194747925    2
194748063    2
194748070    2
194835939    2
Name: class, Length: 202804, dtype: int64, train_mask=txId
3321          True
11108         True
51816         True
68869         True
89273         True
             ...  
194747812    False
194747925    False
194748063    False
194748070    False
194835939    False
Name: Time step, Length: 202804, dtype: bool, test_mask=txId
3321         False
11108        False
51816        False
68869        False
89273        False
             ...  
194747812     True
194747925     True
194748063     True
194748070     True
194835939     True
Name: Time step, Length: 202804, dtype: bool)

In [ ]:
from torch_geometric.transforms import RandomLinkSplit

splitter = RandomLinkSplit(
    is_undirected=False,
    split_labels=True,          
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
    num_val=0,
    num_test=0,
)
train_data, val_data, test_data = splitter(data)
print(train_data.keys())
train_data.pos_edge_label

['train_mask', 'pos_edge_label', 'test_mask', 'y', 'pos_edge_label_index', 'edge_index', 'neg_edge_label_index', 'neg_edge_label', 'x']


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GAE
from sklearn.metrics import f1_score, precision_score, recall_score

# 2) Encoder + Classifier
class GAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

class NodeClassifier(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.lin = torch.nn.Linear(in_channels, num_classes)
    def forward(self, z):
        return self.lin(z)

in_feats    = data.num_node_features
hidden_dim  = 128
out_dim     = 64
num_classes = int(data.y[data.y != 2].max() + 1)

encoder    = GAEEncoder(in_feats, hidden_dim, out_dim)
gae_model  = GAE(encoder)
classifier = NodeClassifier(out_dim, num_classes)

optimizer = torch.optim.Adam(
    list(gae_model.parameters()) + list(classifier.parameters()),
    lr=0.01
)
bce_loss = torch.nn.BCEWithLogitsLoss()
ce_loss  = torch.nn.CrossEntropyLoss()

# 3) train() 改用 train_pos/neg
def train():
    gae_model.train()
    classifier.train()
    optimizer.zero_grad()

    # encode on train positive edges
    z = gae_model.encode(train_data.x, train_data.pos_edge_label_index)

    # AE loss (pos + neg)
    pos = train_data.pos_edge_label_index
    neg = train_data.neg_edge_label_index
    edge_all = torch.cat([pos, neg], dim=1)
    logits_ae = gae_model.decode(z, edge_all)
    labels_ae = torch.cat([
        torch.ones(pos.size(1), device=logits_ae.device),
        torch.zeros(neg.size(1), device=logits_ae.device),
    ])
    loss_ae = bce_loss(logits_ae, labels_ae)

    # classification loss (only known & train_mask)
    # print(len(train_data.y != 2))
    train_mask = torch.tensor(train_data.train_mask.values, dtype=torch.bool)
   
    for i in range(len(train_mask)):
        train_mask[i] = train_mask[i] and (train_data.y[i] != 2)
    mask = train_mask
    #print(mask[mask == True].shape)
    #print(mask.shape)
    logits_cls = classifier(z)
    loss_cls = ce_loss(logits_cls[mask], train_data.y[mask])
    #print(loss_ae)
    loss = loss_ae + loss_cls
    loss.backward()
    optimizer.step()
    return loss.item(), loss_ae.item(), loss_cls.item()

# 4) evaluate() 只用 train_data.train_pos_edge_index encode
def evaluate():
   
    with torch.no_grad():
        z = gae_model.encode(data.x, data.edge_index) 
        logits = classifier(z)
        preds  = logits.argmax(dim=1)
   
        test_mask = torch.tensor(data.test_mask.values, dtype=torch.bool) & (data.y != 2)
  
        mask = test_mask
        y_true = data.y[mask].cpu().numpy()
        y_pred = preds[mask].cpu().numpy()

        acc  = (y_true == y_pred).mean().item()
        prec = precision_score(y_true, y_pred, average='macro')
        rec  = recall_score(y_true, y_pred, average='macro')
        f1   = f1_score(y_true, y_pred, average='macro')
        print(f"Test Acc={acc:.4f} | "
              f"Precision={prec:.4f} | "
              f"Recall={rec:.4f} | "
              f"F1={f1:.4f}")
        return acc

for epoch in range(1, 101):
    loss_tot, loss_ae, loss_cls = train()
    print(f"Epoch {epoch:03d} | "
              f"Loss={loss_tot:.4f} (AE={loss_ae:.4f}, CLF={loss_cls:.4f})")
    if epoch % 10 == 0:
        test_acc= evaluate()

Epoch 001 | Loss=1.5783 (AE=0.7920, CLF=0.7863
Epoch 002 | Loss=1.0372 (AE=0.7822, CLF=0.2551
Epoch 003 | Loss=1.0746 (AE=0.7932, CLF=0.2814
Epoch 004 | Loss=1.0500 (AE=0.7791, CLF=0.2709
Epoch 005 | Loss=1.0047 (AE=0.7617, CLF=0.2430
Epoch 006 | Loss=0.9597 (AE=0.7425, CLF=0.2172
Epoch 007 | Loss=0.9359 (AE=0.7311, CLF=0.2048
Epoch 008 | Loss=0.9278 (AE=0.7254, CLF=0.2024
Epoch 009 | Loss=0.9122 (AE=0.7121, CLF=0.2001
Epoch 010 | Loss=0.8880 (AE=0.6991, CLF=0.1888
Test Acc=0.9088 | Precision=0.6810 | Recall=0.7891 | F1=0.7181
Epoch 011 | Loss=0.8601 (AE=0.6864, CLF=0.1737
Epoch 012 | Loss=0.8522 (AE=0.6864, CLF=0.1657
Epoch 013 | Loss=0.8523 (AE=0.6864, CLF=0.1659
Epoch 014 | Loss=0.8513 (AE=0.6864, CLF=0.1649
Epoch 015 | Loss=0.8429 (AE=0.6864, CLF=0.1565
Epoch 016 | Loss=0.8310 (AE=0.6864, CLF=0.1446
Epoch 017 | Loss=0.8234 (AE=0.6864, CLF=0.1370
Epoch 018 | Loss=0.8220 (AE=0.6864, CLF=0.1356
Epoch 019 | Loss=0.8202 (AE=0.6864, CLF=0.1337
Epoch 020 | Loss=0.8134 (AE=0.6864, CLF=0.12